In [1]:
import pandas as pd
import ezdxf
import numpy as np

In [2]:
path_trm = "../calc/trm_res_plots.xlsx"
path_quad = "../calc/quad_res_plots.xlsx"
path_tho = "../calc/tho_res_plots.xlsx"
sheet_name_trm = ['lpa_0477', 'lpa_0747', 'lpa_0779', 'lpa_0833']
sheet_name_quad = ['lpa_0400', 'lpa_0500']
sheet_name_tho = ['lpa_0633', 'lpa_0568', 'lpa_0366', 'lpa_0300', 'lpa_0101']
sheet_name_tho_def = ['lpa_0633_w', 'lpa_0568_w', 'lpa_0366_w', 'lpa_0300_w', 'lpa_0101_w']

DF_nodes = pd.read_excel(path_tho, sheet_name='knoten')[['X [m]', 'Y [m]', 'Z [m]']]
DF_w = pd.read_excel(path_tho, sheet_name='lpa_0101_w')

In [3]:
DF_w[['Knoten', 'ez  [mm]']].dropna()

,Knoten,ez [mm]
1,-1.0,0.000
11,-85.0,0.000
14,-85.0,0.000
24,-86.0,0.000
27,-86.0,0.000
...,...,...
92220,-378.0,-0.010
92223,-378.0,-0.010
92233,-294.0,-0.005
92236,-294.0,-0.005


In [4]:
def read_data(path, sheet_name):
    DF = pd.read_excel(path, sheet_name=sheet_name)[['Knoten', 'My  [kNm]', 'Tx  [kNm]','Vz  [kN]']].dropna()
    #transformation
    DF['Knoten'] = DF['Knoten'].astype(int)*-1 -1
    #Merge with nodes coordinates
    DF_res = DF.merge(DF_nodes, left_on='Knoten', right_index=True)
    #rename
    columns_rename = ['knoten', 'My', 'Tx','Vz', 'x', 'y', 'z']
    DF_res.columns = columns_rename
    #reset index
    DF_res=DF_res.reset_index()

    return DF_res

def read_data_def(path, sheet_name):
    DF = pd.read_excel(path, sheet_name=sheet_name)[['Knoten', 'ez  [mm]']].dropna()
    #transformation
    DF['Knoten'] = DF['Knoten'].astype(int)*-1 -1
    #Merge with nodes coordinates
    DF_res = DF.merge(DF_nodes, left_on='Knoten', right_index=True)
    #rename
    columns_rename = ['knoten', 'w', 'x', 'y', 'z']
    DF_res.columns = columns_rename
    #reset index
    DF_res=DF_res.reset_index()

    return DF_res

def save_dxf(Data, dxf_name, moment,scale):

    # Extract rows for DXF
    x = Data['x']
    y = Data['y']
    z = Data[moment]/scale

    # Generate consistent pairs (0-1, 2-3, ...)
    index_pairs = [(i, i + 1) for i in range(0, len(x), 2)]

    # Create a new DXF document
    doc = ezdxf.new()

    # Add a new modelspace
    msp = doc.modelspace()



    # Add 3D polyline to the DXF
    points = list(zip(x, y, z))

    # Add line segments for each pair
    for start, end in index_pairs:
        points = [(x[start],y[start],z[start]), (x[end],y[end],z[end])]
        msp.add_polyline3d(points, dxfattribs={'layer': '3D Line'})

    # # Add vertical lines from z=0 to z=z[i] for each point
    # for i in range(len(x)):
    #     base_point = (x[i], y[i], 0)  # Point at z=0
    #     top_point = (x[i], y[i], z[i])  # Point at z=z[i]
    #     msp.add_line(base_point, top_point, dxfattribs={'layer': 'Vertical Lines'})
        

    # Save to file
    output_path = dxf_name+'_'+moment+".dxf"
    doc.saveas(output_path)

    print(f"DXF file saved as {output_path}")

In [5]:
DG_sg = read_data(path = path_tho, sheet_name="lpa_0101")


# Filter nodes with exactly two entries
filtered_nodes = DG_sg.groupby('knoten')

# Group by `knoten` to process pairs
results = []
for knoten, group in filtered_nodes:
    v1, v2 = group['Vz'].iloc[0], group['Vz'].iloc[1]
    
    # Calculate V0 and theta
    V0 = np.sqrt(v1**2 + v2**2)
    theta = np.arctan2(v2, v1)
    
    # Store results
    results.append({'knoten': knoten, 'V0': V0, 'theta': theta, 'theta_deg': np.degrees(theta)})

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

filtered_nodes

IndexError: single positional indexer is out-of-bounds

In [11]:

DG_sg
# # Add results to a new DataFrame (or update the existing one)
# results = pd.DataFrame({'knoten': grouped.groups.keys(), 'V0': V0_list, 'theta': theta_list})
# results['theta_deg'] = np.degrees(results['theta'])  # Optional: Convert to degrees



,index,knoten,My,Tx,Vz,x,y,z
0,1,0,0.000,0.000,0.371,4.3,0.0,0
1,2,84,0.037,0.000,0.371,4.3,0.1,0
2,5,84,-0.037,0.000,0.864,4.3,0.1,0
3,6,85,0.049,0.000,0.864,4.3,0.2,0
4,9,85,-0.049,0.000,1.017,4.3,0.2,0
...,...,...,...,...,...,...,...,...
14187,28374,377,-0.264,0.661,-5.209,8.5,4.0,0
14188,28377,377,0.284,0.568,-4.267,8.5,4.0,0
14189,28378,293,-0.143,0.568,-4.267,8.5,4.1,0
14190,28381,293,0.424,0.000,-4.239,8.5,4.1,0


In [ ]:
# for sheet in sheet_name_tho:    
#     DG_sg = read_data(path = path_tho, sheet_name=sheet)
#     save_dxf(DG_sg, sheet, 'My', scale=5)
#     save_dxf(DG_sg, sheet, 'Tx', scale=5)
#     save_dxf(DG_sg, sheet, 'Vz', scale=20)


# for sheet in sheet_name_tho_def:    
#     DG_w = read_data_def(path = path_tho, sheet_name=sheet)
#     save_dxf(DG_w, sheet, 'w', scale=50)

DXF file saved as lpa_0633_My.dxf
DXF file saved as lpa_0633_Tx.dxf
DXF file saved as lpa_0633_Vz.dxf
DXF file saved as lpa_0568_My.dxf
DXF file saved as lpa_0568_Tx.dxf
DXF file saved as lpa_0568_Vz.dxf
DXF file saved as lpa_0366_My.dxf
DXF file saved as lpa_0366_Tx.dxf
DXF file saved as lpa_0366_Vz.dxf
DXF file saved as lpa_0300_My.dxf
DXF file saved as lpa_0300_Tx.dxf
DXF file saved as lpa_0300_Vz.dxf
DXF file saved as lpa_0101_My.dxf
DXF file saved as lpa_0101_Tx.dxf
DXF file saved as lpa_0101_Vz.dxf
DXF file saved as lpa_0633_w_w.dxf
DXF file saved as lpa_0568_w_w.dxf
DXF file saved as lpa_0366_w_w.dxf
DXF file saved as lpa_0300_w_w.dxf
DXF file saved as lpa_0101_w_w.dxf
